
# Stock Analysis using OpenAI Functions and LangChain Multi Agent

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

**Diclaimer**: This is only a demo notebook of stocks analysis. The recommendations or advise may not be used for actual invesment decision.

In [1]:
!pip install -q yfinance langchain openai langchain_openai langchainhub langsmith httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 175.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00


In [2]:
import openai
import json
import os
import numpy as np
import yfinance as yf
from langchain_core.tools import StructuredTool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from google.colab import userdata

## Defining a custom method for obtaining stats of a stock

**yfinance** is an open source library to access the financial data available on Yahoo Finance, which offers an excellent range of market data on stocks, bonds and currencies along with market news, reports and analysis.

- Define a function which returns total gains, average and standard deviation of daily prices changes of stocks for a specified number of months

In [3]:
def get_stock_stats(ticker: str):

    '''
        input string ticker is The correct ticker symbol for the stock in Nation Stock Exchange (NSE) India.
    '''

    # Get ticker name correctly
    msft = yf.Ticker(ticker.split(".")[0] + ".NS")

    # get historical market data
    hist = msft.history(period = str(3) + "mo" )

    # Compute the market data
    hist['daily_changes']  = (hist['Close'] - hist['Open']) * 100 / hist['Open']

    # Compute different statistics
    total_gain = (hist.iloc[-1]["Close"] - hist.iloc[0]["Open"] ) * 100 / hist.iloc[0]["Open"]
    avg_daily_changes = np.mean(hist['daily_changes'])
    std_daily_changes = np.std(hist['daily_changes'])

    stock_stats = {'total_gain_in_percentage': round(total_gain, 3),
                   'average_daily_changes_in_percentage': round(avg_daily_changes, 3),
                   'std_daily_changes_in_percentage': round(std_daily_changes, 3)}

    return json.dumps(stock_stats)

In [4]:
get_stock_stats("TATAPOWER")

'{"total_gain_in_percentage": 4.69, "average_daily_changes_in_percentage": -0.215, "std_daily_changes_in_percentage": 1.897}'

## Configuring the function as a tool

## Configuring tool as OpenAI function

In [6]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

OpenAI Key:··········


In [ ]:
#os.environ["LANGCHAIN_API_KEY"] = getpass("Langchain API Key:")

Langchain API Key:··········


In [7]:
tool = StructuredTool.from_function(get_stock_stats)


In [21]:
llm = ChatOpenAI(model_name='gpt-4o-2024-08-06',
             temperature=0.0,
             max_tokens = 2048)

In [22]:
tools = [
    tool
]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [23]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke( {"input": "How to calculate risk of a stock?"} )



> Entering new AgentExecutor chain...
The risk of a stock is typically measured by its volatility, which is the degree of variation of a trading price series over time. One common way to calculate the risk of a stock is by using the standard deviation of its historical returns. Here is a general formula to calculate the risk of a stock:

1. Calculate the average return of the stock over a specific period.
2. Calculate the difference between each individual return and the average return.
3. Square each difference.
4. Calculate the average of the squared differences.
5. Take the square root of the average squared differences to get the standard deviation, which represents the risk of the stock.

You can also use statistical measures such as beta, which measures the stock's sensitivity to market movements, to assess the risk of a stock.

If you have a specific stock in mind, I can help you calculate the risk of that stock by providing its historical data and performing the necessary cal

{'input': 'How to calculate risk of a stock?',
 'output': "The risk of a stock is typically measured by its volatility, which is the degree of variation of a trading price series over time. One common way to calculate the risk of a stock is by using the standard deviation of its historical returns. Here is a general formula to calculate the risk of a stock:\n\n1. Calculate the average return of the stock over a specific period.\n2. Calculate the difference between each individual return and the average return.\n3. Square each difference.\n4. Calculate the average of the squared differences.\n5. Take the square root of the average squared differences to get the standard deviation, which represents the risk of the stock.\n\nYou can also use statistical measures such as beta, which measures the stock's sensitivity to market movements, to assess the risk of a stock.\n\nIf you have a specific stock in mind, I can help you calculate the risk of that stock by providing its historical data and

In [24]:
agent_executor.invoke( {"input": "What is the return from TATAPOWER stock in NSE in last three month?"} )



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER'}`


{"total_gain_in_percentage": 4.69, "average_daily_changes_in_percentage": -0.215, "std_daily_changes_in_percentage": 1.897}The return from TATAPOWER stock in the NSE over the last three months is 4.69%.

> Finished chain.


{'input': 'What is the return from TATAPOWER stock in NSE in last three month?',
 'output': 'The return from TATAPOWER stock in the NSE over the last three months is 4.69%.'}

## OpenAI Multi Function Agent

In [25]:
response = agent_executor.invoke( {"input":
    "Between tatapower and adanipower in NSE India, which stock had \
    observerd higher volatility in last one month? Explain the steps for your response. \
    The correct ticket symbol for NSE should be generated from the company names and passed as parameter to tools as necessary."}
)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER'}`
responded: To determine which stock, between Tata Power and Adani Power, had higher volatility in the last month, we need to follow these steps:

1. **Identify the Ticker Symbols**: 
   - Tata Power's ticker symbol on the NSE is "TATAPOWER".
   - Adani Power's ticker symbol on the NSE is "ADANIPOWER".

2. **Fetch Stock Statistics**: 
   - We will use the `get_stock_stats` function to fetch the stock statistics for both "TATAPOWER" and "ADANIPOWER".

3. **Analyze Volatility**:
   - Volatility is typically measured by the standard deviation of the stock's returns over a specific period. In this case, we will look at the volatility over the last month.
   - We will compare the volatility values for both stocks to determine which one had higher volatility.

Let's start by fetching the stock statistics for both companies.

{"total_gain_in_percentage": 4.69, "average_daily_changes_in_percentag

In [26]:
import textwrap

In [27]:
wrapped_text = textwrap.fill(response['output'],
                             width=120,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Based on the stock statistics retrieved:

- **Tata Power (TATAPOWER)**:
  - Standard deviation of daily changes: 1.897%
- **Adani Power (ADANIPOWER)**:
  - Standard deviation of daily changes: 3.336%

### Conclusion:
Adani Power
(ADANIPOWER) had higher volatility in the last month compared to Tata Power (TATAPOWER), as indicated by the higher
standard deviation of daily changes (3.336% vs. 1.897%).

### Explanation:
- **Volatility** is measured by the standard
deviation of daily percentage changes in stock prices. A higher standard deviation indicates higher volatility.
- We
compared the standard deviation of daily changes for both stocks over the last month to determine which one experienced
more fluctuation in its stock price.
